In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "COVID-19"
cohort = "GSE216705"

# Input paths
in_trait_dir = "../../input/GEO/COVID-19"
in_cohort_dir = "../../input/GEO/COVID-19/GSE216705"

# Output paths
out_data_file = "../../output/preprocess/COVID-19/GSE216705.csv"
out_gene_data_file = "../../output/preprocess/COVID-19/gene_data/GSE216705.csv"
out_clinical_data_file = "../../output/preprocess/COVID-19/clinical_data/GSE216705.csv"
json_path = "../../output/preprocess/COVID-19/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Loss of GM-CSF-dependent instruction of alveolar macrophages in COVID-19 provides a rationale for inhaled GM-CSF treatment"
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Refer to individual Series"
Sample Characteristics Dictionary:
{0: ['strain: C57BL/6'], 1: ['metadata info: metaData_microarrays.txt']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# Based on the background information and sample characteristics, let's analyze this dataset

# 1. Gene Expression Data Availability
# The background information about "Loss of GM-CSF-dependent instruction of alveolar macrophages in COVID-19"
# suggests this is likely a gene expression dataset studying COVID-19's effects
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# Looking at the sample characteristics dictionary, we don't see typical human clinical data
# The dict shows 'strain: C57BL/6' which indicates this is likely a mouse model study, not human data
# and 'metadata info: metaData_microarrays.txt' which refers to external metadata

# 2.1 Data Availability
# Since we don't see trait, age, or gender data in the sample characteristics,
# we'll set all row identifiers to None
trait_row = None  # No COVID-19 status information in the sample characteristics
age_row = None    # No age information in the sample characteristics
gender_row = None # No gender information in the sample characteristics

# 2.2 Data Type Conversion
# Define conversion functions in case they're needed, even though we don't have the data
def convert_trait(value):
    if value is None:
        return None
    value = value.split(":")[-1].strip() if ":" in value else value.strip()
    if value.lower() in ["covid-19", "positive", "covid", "yes"]:
        return 1
    elif value.lower() in ["healthy", "control", "negative", "no"]:
        return 0
    return None

def convert_age(value):
    if value is None:
        return None
    value = value.split(":")[-1].strip() if ":" in value else value.strip()
    try:
        return float(value)
    except ValueError:
        return None

def convert_gender(value):
    if value is None:
        return None
    value = value.split(":")[-1].strip() if ":" in value else value.strip()
    if value.lower() in ["female", "f"]:
        return 0
    elif value.lower() in ["male", "m"]:
        return 1
    return None

# 3. Save Metadata
# Trait data is not available since trait_row is None
is_trait_available = trait_row is not None

# Save initial usability information
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is None, we'll skip this substep
# If trait_row was not None, we would have executed:
# clinical_df = geo_select_clinical_features(
#     clinical_df=clinical_data,
#     trait=trait,
#     trait_row=trait_row,
#     convert_trait=convert_trait,
#     age_row=age_row,
#     convert_age=convert_age,
#     gender_row=gender_row,
#     convert_gender=convert_gender
# )
# preview_df(clinical_df)
# clinical_df.to_csv(out_clinical_data_file)


False

### Step 3: Gene Data Extraction

In [4]:
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"SOFT file: {soft_file}")
print(f"Matrix file: {matrix_file}")

# Set gene availability flag
is_gene_available = True  # Initially assume gene data is available

# First check if the matrix file contains the expected marker
found_marker = False
marker_row = None
try:
    with gzip.open(matrix_file, 'rt') as file:
        for i, line in enumerate(file):
            if "!series_matrix_table_begin" in line:
                found_marker = True
                marker_row = i
                print(f"Found the matrix table marker at line {i}")
                break
    
    if not found_marker:
        print("Warning: Could not find '!series_matrix_table_begin' marker in the file.")
        is_gene_available = False
        
    # If marker was found, try to extract gene data
    if is_gene_available:
        try:
            # Try using the library function
            gene_data = get_genetic_data(matrix_file)
            
            if gene_data.shape[0] == 0:
                print("Warning: Extracted gene data has 0 rows.")
                is_gene_available = False
            else:
                print(f"Gene data shape: {gene_data.shape}")
                # Print the first 20 gene/probe identifiers
                print("First 20 gene/probe identifiers:")
                print(gene_data.index[:20].tolist())
        except Exception as e:
            print(f"Error extracting gene data with get_genetic_data(): {e}")
            is_gene_available = False
    
    # If gene data extraction failed, examine file content to diagnose
    if not is_gene_available:
        print("Examining file content to diagnose the issue:")
        try:
            with gzip.open(matrix_file, 'rt') as file:
                # Print lines around the marker if found
                if marker_row is not None:
                    for i, line in enumerate(file):
                        if i >= marker_row - 2 and i <= marker_row + 10:
                            print(f"Line {i}: {line.strip()[:100]}...")
                        if i > marker_row + 10:
                            break
                else:
                    # If marker not found, print first 10 lines
                    for i, line in enumerate(file):
                        if i < 10:
                            print(f"Line {i}: {line.strip()[:100]}...")
                        else:
                            break
        except Exception as e2:
            print(f"Error examining file: {e2}")
        
except Exception as e:
    print(f"Error processing file: {e}")
    is_gene_available = False

# Update validation information if gene data extraction failed
if not is_gene_available:
    print("Gene expression data could not be successfully extracted from this dataset.")
    # Update the validation record since gene data isn't available
    is_trait_available = False  # We already determined trait data isn't available in step 2
    validate_and_save_cohort_info(is_final=False, cohort=cohort, info_path=json_path,
                                 is_gene_available=is_gene_available, is_trait_available=is_trait_available)


SOFT file: ../../input/GEO/COVID-19/GSE216705/GSE216705_family.soft.gz
Matrix file: ../../input/GEO/COVID-19/GSE216705/GSE216705-GPL6246_series_matrix.txt.gz
Found the matrix table marker at line 62
Gene data shape: (35556, 27)
First 20 gene/probe identifiers:
['10338001', '10338002', '10338003', '10338004', '10338005', '10338006', '10338007', '10338008', '10338009', '10338010', '10338011', '10338012', '10338013', '10338014', '10338015', '10338016', '10338017', '10338018', '10338019', '10338020']


### Step 4: Gene Identifier Review

In [5]:
# Analyzing the gene identifiers provided in the previous output
# The identifiers appear to be probe IDs (numeric format like '10338001') rather than standard human gene symbols
# Human gene symbols typically follow patterns like "BRCA1", "TP53", etc.
# These numeric identifiers need to be mapped to human gene symbols for meaningful analysis

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=3))

# Examining the gene_assignment column which appears to contain gene symbol information
print("\nExamining 'gene_assignment' column examples:")
if 'gene_assignment' in gene_annotation.columns:
    # Display a few examples of the gene_assignment column to understand its format
    gene_samples = gene_annotation['gene_assignment'].head(5).tolist()
    for i, sample in enumerate(gene_samples):
        print(f"Example {i+1}: {sample[:200]}..." if isinstance(sample, str) and len(sample) > 200 else f"Example {i+1}: {sample}")
    
    # Check the quality and completeness of the gene_assignment column
    non_null_assignments = gene_annotation['gene_assignment'].notna().sum()
    total_rows = len(gene_annotation)
    print(f"\nGene assignment column completeness: {non_null_assignments}/{total_rows} rows ({non_null_assignments/total_rows:.2%})")
    
    # Check for probe IDs without gene assignments (typically '---' entries)
    missing_assignments = gene_annotation[gene_annotation['gene_assignment'] == '---'].shape[0]
    print(f"Probes without gene assignments: {missing_assignments}/{total_rows} rows ({missing_assignments/total_rows:.2%})")
    
    # Identify the columns needed for gene mapping
    print("\nColumns identified for gene mapping:")
    print("- 'ID': Contains probe IDs (e.g., 7896736)")
    print("- 'gene_assignment': Contains gene information that needs parsing to extract gene symbols")
else:
    print("Error: 'gene_assignment' column not found in annotation data.")
    print("Available columns:", gene_annotation.columns.tolist())



Gene annotation preview:
Columns in gene annotation: ['ID', 'GB_LIST', 'SPOT_ID', 'seqname', 'RANGE_GB', 'RANGE_STRAND', 'RANGE_START', 'RANGE_STOP', 'total_probes', 'gene_assignment', 'mrna_assignment', 'category']
{'ID': ['10344614', '10344616', '10344618'], 'GB_LIST': ['AK145513,AK145782', nan, nan], 'SPOT_ID': ['chr1:3054233-3054733', 'chr1:3102016-3102125', 'chr1:3276323-3277348'], 'seqname': ['chr1', 'chr1', 'chr1'], 'RANGE_GB': ['NC_000067.6', 'NC_000067.6', 'NC_000067.6'], 'RANGE_STRAND': ['+', '+', '+'], 'RANGE_START': ['3054233', '3102016', '3276323'], 'RANGE_STOP': ['3054733', '3102125', '3277348'], 'total_probes': [33.0, 25.0, 25.0], 'gene_assignment': ['ENSMUST00000160944 // Gm16088 // predicted gene 16088 // --- // --- /// ENSMUST00000120800 // Gm14300 // predicted gene 14300 // --- // --- /// ENSMUST00000179907 // G430049J08Rik // RIKEN cDNA G430049J08 gene // --- // --- /// AK145513 // Gm2889 // predicted gene 2889 // 18 A1|18 // 100040658', 'ENSMUST00000082908 // Gm26

### Step 6: Gene Identifier Mapping

In [7]:
# Check the format of IDs in our gene expression data
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
gene_data = get_genetic_data(matrix_file)
print(f"Gene expression data shape: {gene_data.shape}")
print(f"Sample gene expression IDs: {gene_data.index[:5].tolist()}")

# Extract the mapping between probe IDs and gene symbols
# Based on the previous output, we need the 'ID' column (probe identifiers) and 'gene_assignment' column (gene symbols)
print("Creating gene mapping dataframe...")

# Create a function to extract gene symbols from gene_assignment string
def extract_gene_symbols(gene_assignment_str):
    if not isinstance(gene_assignment_str, str) or gene_assignment_str == '---':
        return []
    
    # The format appears to be "ENSMUST... // GeneName // description // --- // ---"
    # We want to extract the gene names (second element after '//')
    genes = []
    assignments = gene_assignment_str.split('///')
    for assignment in assignments:
        parts = assignment.strip().split('//')
        if len(parts) >= 2:
            gene_symbol = parts[1].strip()
            if gene_symbol and gene_symbol != '---':
                genes.append(gene_symbol)
    return genes

# Apply extraction function to create mapping dataframe
gene_annotation['Genes'] = gene_annotation['gene_assignment'].apply(extract_gene_symbols)
valid_rows = gene_annotation['Genes'].apply(len) > 0
mapping_df = gene_annotation.loc[valid_rows, ['ID', 'Genes']]
mapping_df = mapping_df.explode('Genes')
mapping_df = mapping_df.rename(columns={'Genes': 'Gene'})
print(f"Mapping dataframe shape: {mapping_df.shape}")
print("Sample mapping entries:")
print(mapping_df.head())

# Apply gene mapping to convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, mapping_df)
print(f"Gene expression data after mapping: {gene_data.shape}")
print(f"Sample gene symbols: {gene_data.index[:10].tolist()}")

# Save the gene expression data
if gene_data.shape[0] > 0:
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    gene_data.to_csv(out_gene_data_file)
    print(f"Gene expression data saved to {out_gene_data_file}")
else:
    print("No gene expression data to save after mapping.")


Gene expression data shape: (35556, 27)
Sample gene expression IDs: ['10338001', '10338002', '10338003', '10338004', '10338005']
Creating gene mapping dataframe...


Mapping dataframe shape: (290501, 2)
Sample mapping entries:
         ID           Gene
0  10344614        Gm16088
0  10344614        Gm14300
0  10344614  G430049J08Rik
0  10344614         Gm2889
1  10344616        Gm26206


Gene expression data after mapping: (290, 27)
Sample gene symbols: ['A330087I24', 'A730034C02', 'AA066038', 'AA386476', 'AA388235', 'AA414768', 'AA415398', 'AA467197', 'AA474408', 'AA667203']
Gene expression data saved to ../../output/preprocess/COVID-19/gene_data/GSE216705.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")

# Create output directory if it doesn't exist
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)

# Save the normalized gene data
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# Determine if trait data is available
is_trait_available = trait_row is not None

# Skip clinical processing if trait_row is None
if is_trait_available:
    # 2. Extract clinical features using the previously identified feature rows
    clinical_features = geo_select_clinical_features(
        clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Create directory for clinical data output
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Save the clinical features
    clinical_features.to_csv(out_clinical_data_file)
    print(f"Clinical features saved to {out_clinical_data_file}")
    
    # Preview the clinical features
    clinical_features_preview = preview_df(clinical_features.T)
    print("Clinical features preview:")
    print(clinical_features_preview)
    
    # 3. Link clinical and genetic data
    linked_data = geo_link_clinical_genetic_data(clinical_features, normalized_gene_data)
    print(f"Linked data shape: {linked_data.shape}")
    
    # 4. Handle missing values in the linked data
    linked_data = handle_missing_values(linked_data, trait)
    print(f"Linked data shape after handling missing values: {linked_data.shape}")
    
    # 5. Determine if trait and demographic features are biased
    is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
else:
    print("No clinical features available, skipping clinical data processing.")
    # Create a minimal DataFrame with the trait column
    linked_data = pd.DataFrame({trait: []})
    is_biased = True  # Set to True because without trait data, it's unusable

# 6. Validate and save cohort info
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available,
    is_biased=is_biased,
    df=linked_data,
    note="Dataset contains mouse gene expression data but lacks human clinical annotations for COVID-19."
)

# 7. Save the linked data if it's usable
if is_usable and is_trait_available:
    # Create output directory if it doesn't exist
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    
    # Save the linked data
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Linked data not saved due to quality issues or missing trait information.")

Gene data shape after normalization: (34, 27)
Normalized gene data saved to ../../output/preprocess/COVID-19/gene_data/GSE216705.csv
No clinical features available, skipping clinical data processing.
Abnormality detected in the cohort: GSE216705. Preprocessing failed.
Linked data not saved due to quality issues or missing trait information.
